In [38]:
import random
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.models import load_model
from keras.layers import Flatten, Dense, Concatenate, Dot, Lambda, Input,Conv2D,MaxPool2D,Dropout
from keras.models import model_from_json
from keras.datasets import mnist
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from keras import backend as K
import os
import cv2
import glob
import pandas as pd
from sklearn.utils import shuffle


In [39]:
class PreProcess():
    def __init__(self,image_file_name):
        self.image_file_name = image_file_name
        self.data_frame=None
        self.label_mapping = self.genrate_mapping()
        
    def genrate_mapping(self):
        mapping={}
        for i,v in enumerate(os.listdir(self.image_file_name)):
            mapping[v]=i
        return mapping
    
    def get_image_file(self,size=64,sample=100):
        root_file = os.path.abspath(self.image_file_name)
        list_of_file = os.listdir(self.image_file_name)
        df_img = []
        df_label = []
        for x in list_of_file:
            path = os.listdir(os.path.join(root_file,x))
            c=0
            for pt in path:
                try:
                    img_path = os.path.join(root_file,os.path.join(x,pt))
                    image = cv2.imread(img_path,0)
                    image = cv2.resize(image,(size,size))
                    df_img.append(list(image.reshape(1,-1)[0]))
                    df_label.append(self.label_mapping[x])                    
                except Exception as e:
                    print("Corrupted data")
                #just comment below code to convert everything to mnist
                #--------------------------------
                if(c==sample):
                    break
                c+=1
                #--------------------------------
                
        cvt_df={}
        col=0
        for i in range(len(df_img[0])):
            img_pixil_list=[]
            for j in range(len(df_img)):
                img_pixil_list.append(df_img[j][i])
            cvt_df["pixil"+str(col)]=img_pixil_list
            col+=1
            
        cvt_df['label'] = df_label
        self.data_frame= pd.DataFrame(cvt_df)
        self.data_frame = self.data_frame.sample(frac=1)
        return self.data_frame
    
    def save_minist_file(self):
        try:
            print("image converted")
            self.data_frame.to_csv("plant_minist.csv",index=False)
        except Exception as e:
            print("data not there")

In [40]:
class SiameseModel():
    def __init__(self,csv_file_name,size):
        self.csv_file_name = csv_file_name
        self.size = size
        self.model = None
        self.pairs_train = None
        self.pairs_test = None
        self.labels_train = None
        self.labels_test = None
        self.y_pred=None
        keras.backend.clear_session()
        
    def make_pairs(self,x, y):
        num_classes = max(y) + 1
        number_indices = [np.where(y == i)[0] for i in range(num_classes)]
        pairs = []
        labels = []

        for idx1 in range(len(x)):
            # add a matching example
            print("pair genrated "+str(idx1))
            x1 = x[idx1]
            label1 = y[idx1]
            idx2 = random.choice(number_indices[label1])
            x2 = x[idx2]

            pairs += [[x1, x2]]
            labels += [1]

            # add a not matching example
            label2 = random.randint(0, num_classes-1)
            while label2 == label1:
                label2 = random.randint(0, num_classes-1)

            idx2 = random.choice(number_indices[label2])
            x2 = x[idx2]

            pairs += [[x1, x2]]
            labels += [0]
            for i in range(len(pairs)):
                for j in range(len(pairs[i])):
                    pairs[i][j] = pairs[i][j].reshape(self.size,self.size)
                    
        return np.array(pairs), np.array(labels)
    
    def train_model(self,epochs=10):
        try:
            self.model.fit([self.pairs_train[:,0], self.pairs_train[:,1]], self.labels_train[:], batch_size=16, epochs=epochs)
            self.model.save_weights('model_weights.h5')
                
        except Exception as e:
            print("No Model Initialize")
            
    def predict_similarity(self):
        self.model.load_weights('model_weights.h5')
        
        self.y_pred = self.model.predict([self.pairs_test[:,0],self.pairs_test[:,1]])
        return self.y_pred
    
    def euclidean_distance(self,vects):
        x, y = vects
        sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
        return K.sqrt(K.maximum(sum_square, K.epsilon()))
    
    def create_model(self,input_shape):
        i = input_shape[2]
        j = input_shape[3]
        inp = Input(shape=(i,j))
        x = Flatten()(inp)
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.1)(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.1)(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.1)(x)
        x = Dense(256, activation='relu')(x)
        x = Dropout(0.1)(x)
        x = Dense(4096, activation='sigmoid')(x)
        dense = Model(inp, x)

        input1 = Input((i,j))
        input2 = Input((i,j))

        dense1 = dense(input1)
        dense2 = dense(input2)

        merge_layer = Lambda(self.euclidean_distance)([dense1,dense2])
        dense_layer = Dense(1, activation="sigmoid")(merge_layer)
        model = Model(inputs=[input1, input2], outputs=dense_layer)
        model.compile(loss = "binary_crossentropy", optimizer="adam", metrics=["accuracy"])
        return model
    
    def process_data(self):
        data = pd.read_csv(self.csv_file_name)
        labels = data['label'].values
        images = data.drop(['label'],axis=1).values
        x_train,x_test, y_train, y_test = train_test_split(images,labels,test_size=0.2,random_state=0)
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train /= 255
        x_test /= 255
        self.pairs_train, self.labels_train = self.make_pairs(x_train, y_train)
        self.pairs_test, self.labels_test = self.make_pairs(x_test, y_test)
        
        self.model = self.create_model(self.pairs_train.shape)
        

In [41]:
if __name__ == "__main__":
    #just delete the plant_mnist.csv file to generate it again
    files = os.listdir()
    #decide number of epochs
    ephs = 50
    if("plant_minist.csv" not in files):
        try:
            try:
                model = PreProcess("PlantVillage")
                #give manunal image mnist file you want to create and number of file you want to feed
                #size = 64
                #samples=100
                data = model.get_image_file(64,100)
                #save the file
                model.save_minist_file()
            except Exception as e:
                print(e)
            #to stop precomputation of the file, if it is present there
            sam = SiameseModel("plant_minist.csv",64)
            #put number of epochs
            sam.process_data()
            sam.train_model(ephs)
            predictions=sam.predict_similarity()
        except Exception as e:
            print(e)
    else:
        try:
            sam = SiameseModel("plant_minist.csv",64)
            sam.process_data()
            #put number of epochs
            sam.train_model(ephs)
            predictions=sam.predict_similarity()
        except Exception as e:
            print(e)
    

pair genrated 0
pair genrated 1
pair genrated 2
pair genrated 3
pair genrated 4
pair genrated 5
pair genrated 6
pair genrated 7
pair genrated 8
pair genrated 9
pair genrated 10
pair genrated 11
pair genrated 12
pair genrated 13
pair genrated 14
pair genrated 15
pair genrated 16
pair genrated 17
pair genrated 18
pair genrated 19
pair genrated 20
pair genrated 21
pair genrated 22
pair genrated 23
pair genrated 24
pair genrated 25
pair genrated 26
pair genrated 27
pair genrated 28
pair genrated 29
pair genrated 30
pair genrated 31
pair genrated 32
pair genrated 33
pair genrated 34
pair genrated 35
pair genrated 36
pair genrated 37
pair genrated 38
pair genrated 39
pair genrated 40
pair genrated 41
pair genrated 42
pair genrated 43
pair genrated 44
pair genrated 45
pair genrated 46
pair genrated 47
pair genrated 48
pair genrated 49
pair genrated 50
pair genrated 51
pair genrated 52
pair genrated 53
pair genrated 54
pair genrated 55
pair genrated 56
pair genrated 57
pair genrated 58
pair ge

pair genrated 504
pair genrated 505
pair genrated 506
pair genrated 507
pair genrated 508
pair genrated 509
pair genrated 510
pair genrated 511
pair genrated 512
pair genrated 513
pair genrated 514
pair genrated 515
pair genrated 516
pair genrated 517
pair genrated 518
pair genrated 519
pair genrated 520
pair genrated 521
pair genrated 522
pair genrated 523
pair genrated 524
pair genrated 525
pair genrated 526
pair genrated 527
pair genrated 528
pair genrated 529
pair genrated 530
pair genrated 531
pair genrated 532
pair genrated 533
pair genrated 534
pair genrated 535
pair genrated 536
pair genrated 537
pair genrated 538
pair genrated 539
pair genrated 540
pair genrated 541
pair genrated 542
pair genrated 543
pair genrated 544
pair genrated 545
pair genrated 546
pair genrated 547
pair genrated 548
pair genrated 549
pair genrated 550
pair genrated 551
pair genrated 552
pair genrated 553
pair genrated 554
pair genrated 555
pair genrated 556
pair genrated 557
pair genrated 558
pair genra

pair genrated 991
pair genrated 992
pair genrated 993
pair genrated 994
pair genrated 995
pair genrated 996
pair genrated 997
pair genrated 998
pair genrated 999
pair genrated 1000
pair genrated 1001
pair genrated 1002
pair genrated 1003
pair genrated 1004
pair genrated 1005
pair genrated 1006
pair genrated 1007
pair genrated 1008
pair genrated 1009
pair genrated 1010
pair genrated 1011
pair genrated 1012
pair genrated 1013
pair genrated 1014
pair genrated 1015
pair genrated 1016
pair genrated 1017
pair genrated 1018
pair genrated 1019
pair genrated 1020
pair genrated 1021
pair genrated 1022
pair genrated 1023
pair genrated 1024
pair genrated 1025
pair genrated 1026
pair genrated 1027
pair genrated 1028
pair genrated 1029
pair genrated 1030
pair genrated 1031
pair genrated 1032
pair genrated 1033
pair genrated 1034
pair genrated 1035
pair genrated 1036
pair genrated 1037
pair genrated 1038
pair genrated 1039
pair genrated 1040
pair genrated 1041
pair genrated 1042
pair genrated 1043
pa

pair genrated 285
pair genrated 286
pair genrated 287
pair genrated 288
pair genrated 289
pair genrated 290
pair genrated 291
pair genrated 292
pair genrated 293
pair genrated 294
pair genrated 295
pair genrated 296
pair genrated 297
pair genrated 298
pair genrated 299
pair genrated 300
pair genrated 301
pair genrated 302
Epoch 1/1
2424/2424 [==============================] - 11s 5ms/step - loss: 0.7005 - accuracy: 0.4884


In [42]:
predictions=sam.predict_similarity()
print("Pairs"+str(0)+" :",predictions[0])

Pairs0 : [0.49629295]
